In [398]:
import pandas as pd
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
options = Options()
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
driver = webdriver.Chrome(options=options)

In [400]:
arsenal_url = 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats'

In [401]:
data = requests.get(arsenal_url)

In [402]:
from bs4 import BeautifulSoup

In [403]:
soup = BeautifulSoup(data.text)

In [404]:
years = list(range(2024, 2022, -1))
years

[2024, 2023]

In [405]:
keepers=[]
defenders=[]
midfielders=[]
attackers=[]
all_positions = [keepers, defenders, midfielders, attackers]
type(all_positions)


list

In [406]:
for year in years:
    driver.get(arsenal_url)    
    time.sleep(2)

    try:
        driver.find_element(By.CSS_SELECTOR, 'button[mode=primary]').click()
    except Exception:
        pass

    previous_season = soup.select("div.prevnext a")[0].get("href")
    arsenal_url = f"https://fbref.com/{previous_season}"

    for position in all_positions:
        value = ""
        if position == keepers:
            value = "keeper_adv"
        elif position == defenders:
            value = "defense"
        elif position == midfielders:
            value = "passing"
        elif position == attackers:
            value = "shooting"
        else:
            value = "keeper"
        time.sleep(2)

        print(value)

        togglep90 =  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"button[id*='stats_{value}_']")))
        tablename = driver.find_element(By.ID, f"all_stats_{value}")
        driver.execute_script("arguments[0].scrollIntoView();", tablename)

        time.sleep(2)
        togglep90.click()

        page_source = BeautifulSoup(driver.page_source, 'html.parser')
        tables = page_source.findAll('table', id=lambda x: x and x.startswith(f'stats_{value}'))[0]
        Actions = pd.read_html(str(tables))[0]
        Actions["Season"] = year #adding season column
        position.append(Actions)

driver.quit()


keeper_adv


C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36376\3302038101.py:38: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  Actions = pd.read_html(str(tables))[0]


defense


C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36376\3302038101.py:38: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  Actions = pd.read_html(str(tables))[0]


passing


...

C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36376\3302038101.py:38: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  Actions = pd.read_html(str(tables))[0]


passing


C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36376\3302038101.py:38: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  Actions = pd.read_html(str(tables))[0]


shooting


C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_36376\3302038101.py:38: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  Actions = pd.read_html(str(tables))[0]


In [407]:
GKdf = pd.concat(keepers)
DEFdf= pd.concat(defenders)
MIDdf = pd.concat(midfielders)
ATTdf = pd.concat(attackers)

all_df = [GKdf, DEFdf, MIDdf, ATTdf]

In [408]:
for df in all_df:
    df.columns = ['_'.join(col).strip() for col in df.columns.values]

In [409]:
all_df = [GKdf, DEFdf, MIDdf, ATTdf]

ATTdf = ATTdf.reset_index()
MIDdf = MIDdf.reset_index()
DEFdf = DEFdf.reset_index()
GKdf = GKdf.reset_index()

ATTdf = ATTdf.drop('index', axis=1)
MIDdf = MIDdf.drop('index', axis=1)
DEFdf = DEFdf.drop('index', axis=1)
GKdf = GKdf.drop('index', axis=1)

ATTdf = ATTdf.set_index('Unnamed: 0_level_0_Player')
MIDdf = MIDdf.set_index('Unnamed: 0_level_0_Player')
DEFdf = DEFdf.set_index('Unnamed: 0_level_0_Player')
GKdf = GKdf.set_index('Unnamed: 0_level_0_Player')

ATTdf = ATTdf.drop(["Squad Total", "Opponent Total"])
MIDdf = MIDdf.drop(["Squad Total", "Opponent Total"])
DEFdf = DEFdf.drop(["Squad Total", "Opponent Total"])
GKdf = GKdf.drop(["Squad Total", "Opponent Total"])

ATTdf = ATTdf.reset_index()
MIDdf = MIDdf.reset_index()
DEFdf = DEFdf.reset_index()
GKdf = GKdf.reset_index()

In [410]:
ATTdf = ATTdf[ATTdf["Unnamed: 2_level_0_Pos"].str.contains("FW")]
MIDdf = MIDdf[MIDdf["Unnamed: 2_level_0_Pos"].str.contains("MF")]
DEFdf = DEFdf[DEFdf["Unnamed: 2_level_0_Pos"].str.contains("DF")]


ATTdf = ATTdf.reset_index()
MIDdf = MIDdf.reset_index()
DEFdf = DEFdf.reset_index()

ATTdf = ATTdf.drop("index", axis=1)
MIDdf = MIDdf.drop("index", axis=1)
DEFdf = DEFdf.drop("index", axis=1)




In [411]:
ATTdf = ATTdf.drop('Unnamed: 22_level_0_Matches', axis=1)
MIDdf = MIDdf.drop('Unnamed: 28_level_0_Matches', axis=1)
DEFdf = DEFdf.drop(["Challenges_Tkl", "Challenges_Att", "Challenges_Tkl%", "Challenges_Lost", "Unnamed: 21_level_0_Matches"], axis=1)
GKdf = GKdf.drop(["Goals_FK", "Goal Kicks_Att", "Crosses_Opp", "Crosses_Stp", "Expected_/90", "Unnamed: 30_level_0_Matches"], axis=1)


In [412]:
GKdf.columns

Index(['Unnamed: 0_level_0_Player', 'Unnamed: 1_level_0_Nation',
       'Unnamed: 2_level_0_Pos', 'Unnamed: 3_level_0_Age',
       'Unnamed: 4_level_0_90s', 'Goals_GA', 'Goals_PKA', 'Goals_CK',
       'Goals_OG', 'Expected_PSxG', 'Expected_PSxG/SoT', 'Expected_PSxG+/-',
       'Launched_Cmp', 'Launched_Att', 'Launched_Cmp%', 'Passes_Att (GK)',
       'Passes_Thr', 'Passes_Launch%', 'Passes_AvgLen', 'Goal Kicks_Launch%',
       'Goal Kicks_AvgLen', 'Crosses_Stp%', 'Sweeper_#OPA', 'Sweeper_#OPA/90',
       'Sweeper_AvgDist', 'Season_'],
      dtype='object')

In [413]:
ATTdf.columns = ['Player', 'Nation', 'Position', 'Age', '90s', 'Goals', 'Shots', 'SOT',
'SOT%', 'Sh/90', 'SOT/90', 'G/Sh', 'G/SOT', 'Dist', 'FK', 'PK', 'PKatt', 'xG', 'npxG', 'npXg/Sh', 
'G-xG', 'np:g-xG', 'Season']

MIDdf.columns = ['Player', 'Nation', 'Position', 'Age', '90s', 'PassCmp', 'PassAtt', 'PassComp%',
'PassTotDist', 'PassPrgDist', 'ShortPassCmp', 'ShortPassAtt', 'ShortPassCmp%', 'MediumPassCmp', 'MediumPassAtt', 'MediumPassCmp%',
'LongPassCmp', 'LongPassAtt', 'LongPassCmp%','Assists', 'xAG', 'xAssists', 'A-xAG', 'KeyPass', 'IntoFin1/3', 'PassPenArea',
'Crosses', 'ProgPass', 'Season']

DEFdf.columns = ['Player', 'Nation', 'Position', 'Age', '90s', 'Tackles', 'TacklesWon', 'TacklesDef1/3', 'TacklesMid1/3', 'TacklesAtt1/3',
'Blocks', 'ShotsBlocked', 'PassesBlocked', 'Interceptions', 'Tkl+Int', 'Clearances', 'ErrorsToShot', 'Season']

GKdf.columns = ['Player', 'Nation', 'Position', 'Age', '90s', 'GA', 'PKGA', 'CKGA' ,'OG', 'PSxG', 'PSxG/SoT',
'PSxG+/-GA', 'LaunchCmp', 'LaunchAtt', 'LaunchCmp%', 'PassesAtt', 'ThrowsAtt', 'Launch%', 'AvgPassLen', 'GKLaunch%',
'AvgGKLen', 'CrossesStop%', 'OPA', 'OPA/90', 'AvgDistActions', 'Season']

In [414]:
GKdf

,Player,Nation,Position,Age,90s,GA,PKGA,CKGA,OG,PSxG,...,ThrowsAtt,Launch%,AvgPassLen,GKLaunch%,AvgGKLen,CrossesStop%,OutsidePenActions,OutsidePenActions/90,AvgDistActions,Season
0,Guglielmo Vicario,it ITA,GK,27-216,35.0,1.66,0.17,0.23,0.09,1.62,...,5.17,15.0,24.4,14.8,22.1,6.1,2.23,2.23,17.3,2024
1,Hugo Lloris,fr FRA,GK,35.0,24.5,1.59,0.04,0.12,0.04,1.38,...,4.45,31.7,31.4,25.3,25.8,6.2,1.18,1.18,13.7,2023
2,Fraser Forster,eng ENG,GK,34.0,13.5,1.78,0.15,0.30,0.00,1.58,...,5.26,29.0,30.2,49.5,37.6,2.6,1.19,1.19,15.2,2023


In [415]:
GKdf.to_csv("SpursKeepers.csv")
DEFdf.to_csv("SpursDefenders.csv")
MIDdf.to_csv("SpursMidfielders.csv")
ATTdf.to_csv("SpursAttackers.csv")